In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()
conf.set("spark.app.name", "Dmitriy Sokolov Spark RDD app")

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("test")
         .getOrCreate())


In [3]:
spark

In [4]:
sc

<SparkContext master=yarn appName=pyspark-shell>

In [5]:
!hdfs dfs -ls /labs/laba01/ml-100k/u.data

rdd = sc.textFile("/labs/laba01/ml-100k/u.data")

-rw-r--r--   3 hdfs hdfs    1979173 2022-01-06 18:46 /labs/laba01/ml-100k/u.data


In [6]:
rdd.take(10)

['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596',
 '298\t474\t4\t884182806',
 '115\t265\t2\t881171488',
 '253\t465\t5\t891628467',
 '305\t451\t3\t886324817',
 '6\t86\t3\t883603013']

In [7]:
def split_and_edit(line):
    split = line.split("\t")
    return split[2], split[1]

In [8]:
u_data = rdd.map(split_and_edit)

In [9]:
u_data.take(5)

[('3', '242'), ('3', '302'), ('1', '377'), ('2', '51'), ('1', '346')]

In [10]:
u_data_filter = u_data.filter(lambda x: x[1] == "22")

In [11]:
u_data_filter

PythonRDD[4] at RDD at PythonRDD.scala:53

In [12]:
u_data_filter.take(5)

[('1', '22'), ('5', '22'), ('4', '22'), ('5', '22'), ('3', '22')]

In [16]:
gr_u_data_filter = list(u_data_filter.sortByKey(True).countByKey().items())
gr_u_data_filter

[('1', 5), ('2', 14), ('3', 46), ('4', 98), ('5', 134)]

In [15]:
gr_u_data = list(u_data.sortByKey(True).countByKey().items())
gr_u_data

[('1', 6110), ('2', 11370), ('3', 27145), ('4', 34174), ('5', 21201)]

In [18]:
def get_second_el(row):
    return row[1]

In [19]:
import json
import pandas as pd

In [24]:
res = json.dumps({
    "hist_film" : list(map(get_second_el, gr_u_data_filter)), 
     "hist_all" : list(map(get_second_el, gr_u_data))
})

In [25]:
res

'{"hist_film": [5, 14, 46, 98, 134], "hist_all": [6110, 11370, 27145, 34174, 21201]}'

In [26]:
with open('lab01.json', 'w') as outfile:
    json.dump(res, outfile)

In [28]:
spark.stop()